S1: What OpenClaw is

OpenClaw is an FOSS, self-hosted AI gateway and personal assistant framework. 
Think of it as a local WebSocket server that sits in the middle between:
   - Messaging Platforms
   - AI agents

It gives agents the ability to:
   - Receive and send messages across any chat platform
   - Execute tools (browser automation, bash commands, file operations, memory
     search)
   - Manage sessions and long-term memory
   - Run inside sandboxed Docker containers


WHY THIS MATTERS FOR YOUR PROJECT: OpenClaw is essentially a working blueprint
for giving an AI agent the infrastructure to operate semi-autonomously across
multiple channels. It has the tool system, session persistence, multi-provider
support, and plugin architecture that your "New Dejima" agents would need.



---
SECTION 2: THE ENTRY POINT & HOW IT WORKS

The boot chain is simple:
   openclaw.mjs -> dist/entry.js -> src/index.ts -> CLI program (Commander.js)

- openclaw.mjs - The file you run. It just loads the compiled distribution.
- src/index.ts - Builds the CLI command tree using Commander.js with 100+
  command handlers.
- The main command is `gateway` which starts the WebSocket control plane on port
  18789.


WHY THIS MATTERS FOR YOU: Your agent loop system needs an entry point like this.
When an agent "Wakes up", something needs to bootstrap the environment, load
tools, and start the main loop. OpenClaw's boot chain is a clean reference for
that.



---
SECTION 3: THE GATEWAY (`src/gateway/`, ~130 files)

This is the BRAIN of OpenClaw. The Gateway is a WebSocket server that handles:
   - SESSION MANAGEMENT - Each conversation gets an isolated session (main
     sessions for 1-on-1, group sessions for groups)
   - CHANNEL ROUTING - Incoming messages get routed to the right agent based on
     rules.
   - CLIENT MANAGEMENT - WebSocket connection lifecycle, authentication.
   - CONFIGURATION - Config reloading, state persistence
   - PRESENCE & TYPING - Real-time status indicators
   - AGENT INTEGRATION - Connects to the Pi agent via RPC.

WHY THIS MATTERS: Your agent consciousness loop needs soemthing like this 
   Gateway. it's the central control plane that coordinates everything. Your 
   agent needs a persistent process that manages their state, routes their 
   actions, and coordinates their tool usage.


---   
SECTION 4: THE AGENT RUNTIME (`src/agents/`, ~300+ files)

This is where agents actually THINK AND ACT:
   - Pi Embedded Runner (`pi-embedded-runner.ts`) - Runs agent inference 
        in-process, manages session state
   - Pi Embedded Subscribe (`pi-embedded-subscribe.ts`) - Streams responses with
        tool events
   - Model Selection (`model-selection.ts`, `model-fallback.ts`) - Multi-model
        failover
   - Auth Profiles - OAuth token rotation across providers
   - Tool Management - 60+ tool definitions (browser, bash, canvas)
   - Skills - Workspace/managed/bundled skill loading and execution
   - Subagents - Multi-agent routing and cross-agent message spawning
   - System Prompts - Dynamic prompt building per agent/session


CRITICAL FOR YOUR PROJECT:   
   This is the core of what you need to replicate. The agent runtime handles:
      1. Making LLM API calls with streaming
      2. Tool execution (the agent calling bash, browsing web, etc.)
      3. Multi-model failover (if one provider fails, try another)
      4. Subagent spwaning (one agent creating another - your "reproduction"
         system)




---
SECTION 5: THE TOOL SYSTEM

OpenClaw gives agents 60+ BUILT-IN TOOLS:
   - BROWSER - Playwright-based web automation
   - BASH - Shell command execution
   - CANVAS - Interactive UI rendering
   - MEMORY SEARCH - Vector search over past conversations
   - FILE OPERATIONS - Read, write, edit files
   - SESSION MANAGEMENT - Create/switch between sessions

Tools have a POLICY ENFORCEMENT layer - per-agent and per-session restrictions
on what tools an agent can use. There's also an APPROVAL WORKFLOW where 
dangerous commands require human approval.

CRITICAL FOR YOUR PROJECT: Your agents need tools to make money. OpenClaw's
tool system shows you how to:
   - Define tools with schemas
   - Execute them safely in sandboxes
   - Enforce policies on what agents can/can't do
   - Track tool calls with IDs for auditing

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣠⣤⣤⣤⣤⣤⣤⣄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⣠⠾⠛⠉⠉⠉⠁⠀⠀⠉⠉⠉⠉⠛⠷⣄⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢸⡏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⡇⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢸⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣸⠇⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠹⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠏⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⣄⠀⠀⠀⠀⠀⠀⠀⠀⢠⡿⠃⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⡄⠀⠀⠀⠀⠀⠀⢀⡿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣷⡀⠀⠀⠀⠀⢀⣾⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣷⡀⢀⠀⢀⣾⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣗⠒⠨⢥⣾⣦⠬⠑⠒⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣸⣤⣄⣸⠏⢿⣀⣤⣴⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣿⣍⡉⠁⠉⣉⣽⣿⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡸⢀⣼⣁⣠⣀⣹⣄⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠷⢾⣿⠿⠉⠹⢿⣿⡶⠷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀


---   
SECTION 6: THE PLUGIN & EXTENSION SYSTEM (`extension/`, 30+ plugins)

OpenClaw uses a plugin architecture for extensibility:
   - Each plugin lives in `extensions/` as its own workspace package
   - Has a manifest file `openclaw.plugin.json` defining its ID, channels, and
     config schema
   - Plugins are lazy-loaded and validated with Zod schemas.
   - They can provide: new channels, auth providers, tool slots, hooks


EXISTING PLUGINS INCLUDE: Discord, Telegram, Slack, Signal, iMessage...

WHY THIS MATTERS: Your "New Dejima port system" - the interfaces that let agents
   interact with the real world - this IS the plugin system. Each new 
   marketplace, exchange, or service you want agents to access becomes a plugin/
   extension.




---   
SECTION 7: MEMORY & SEARCH (`src/memory/`)

OpenClaw has a sophisticated long-term memory system:
   - QMD Manager        - 

In [ ]:
⣤⣤⣤⣤⣤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣿⣿⠭⠏⠉⣷⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀M
⣿⣿⡁⠠⢀⠉⠉⣷⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀I
⠀⢰⣿⡇⠀⡄⠁⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀N
⠀⠀⠉⢹⣿⡇⠀⡁⠂⠉⢹⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ E
⠀⠀⠀⠀⠀⠿⠿⣅⣀⠉⠂⠀⠿⣿⣁⣀⠀⠀⠀⠀⠀⢀⣀⣀⣀⠀⠀⠀⠀C
⠀⠀⠀⠀⠀⠀⠀⠿⠿⣀⣌⠐⠀⡀⠿⠿⣀⣀⠀⢀⣀⣿⣿⣿⣿⠀⠀⠀⠀R
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠿⢷⣄⡂⠄⠠⠀⠿⢿⣀⣼⠿⣿⣿⠿⠿⠀⠀⠀⠀A
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⣧⣤⡆⠁⠄⢸⣿⣏⢣⣿⣿⠀⠀⠀⠀⠀⠀F
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⣷⣶⣾⣾⡛⣿⣶⡏⠉⠀⠀⠀⠀⠀⠀T
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣲⣶⣿⣿⠹⣿⣿⣿⣿⣷⣶⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣤⣿⣿⣿⣿⣿⣿⠛⢻⣿⣿⣿⣤⣤⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⡋⠉⠉⠉⠀⠈⠉⢿⣿⣿⣿⣶⣶⣶⡆
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⣿⣽⣿⣿⠁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡿⠿⢿⣿⣿⠇

OpenClaw is an FOSS, self-hosted AI gateway and personal assistant framework.
Think of it as a local WebSocket server that sits in middle between:
   - Messaging Platforms
   - AI Agents


It gives AI agents the ability to:   
- Receive and send messages across any chat platform
- Execute tools (browser automation, bash commands, file operations, memory
  search)
- Manage sessions and long-term memory
- Run inside sandboxed Docker containers


WHY THIS MATTERS FOR YOUR PROJECT: OpenClaw is essentially a working blueprint
for giving an AI agent the infrastructure to operate semi-autonomously across
multiple channels. It has the tool system, session persistence, multi-provider
support, and plugin architecture that your "New Dejima" agents would need.




<<<<>>>>
SECTION 2: THE ENTRY POINT & HOW IT BOOTS

The boot chain is simple:

   openclaw.mjs -> dist/entry.js -> src/index.ts -> CLI program (Commander.js)

- openclaw.mjs - The file you run. It just loads the compiled distribution.
- src/index.ts - Builds the CLI command tree using Commander.js with 100+ 
  command handlers.              <-- wow! all the handlers!!!
- The main command in `gateway` which starts the WebSocket control plane on port
  18789.

WHY THIS MATTERS FOR YOU: Your agent loop system needs an entry point like this.
   When an agent "wakes up", something needs to bootstrap the environment, load
   tools, and start the main loop. OpenClaw's boot chain is a clean reference
   for that.




<<<<>>>>   
SECTION 3: THE GATEWAY (`src/gateway/`, ~130 files)

This is the BRAIN of OpenClaw, The Gateway is a WebSocket server that handles
(main sessions for 1-on-1, group sessions for groups):
   - Channel routing - Incoming messages get routed to the first agent based on
     rules
   - Client management - WebSocket connection lifecycle, authentication  
   - Configuration - Config reloading, state persistence
   - Presence & Typing - Real-time status indicators
   - Agent Integration - Connects to the Pi agent runtime via RPC.

WHY THIS MATTERS: Your agent consciousness loop needs something like this 
Gateway. It's the central control plane that coordinates everything. Your agents
need a persistent process that manages their state, routes their actions, and
coordinates their tool usage.





<<<<>>>>
SECTION 4: THE AGENT RUNTIME (`src/agents/`, ~300+ files)
This is where agents actually THINK AND ACT:
   - Pi Embedded Runner (pi-embedded-runner.ts)
   - Pi Embedded Subscribe (`pi-embedded-subscribe.ts`) - Streams responses with
     tool events
   - Model Selection (`model-selection.ts`, `model-fallback.ts`) - Multi-model
     failover                          <-- wow now that i think about it... this is
                                          insane!! from a innovation perspective tbf!!
                                          as well as just integration in general!!
   - Auth Profiles - OAuth token rotation across providers (Anthropic, OpenAI, 
     etc.)    
   - Tool Management - 60+ tool definitions (browser, bash, canvas, memory 
     search)
   - Skills - Workspace/management/bundled skill loading and execution
   - Subagents - Multi-agent routing and cross-agent message spwaning
   - System Prompts - Dynamic prompt building per agent/session



CRITICAL FOR YOUR PROJECT: This is the core of what you need to replicate. The
agent runtime handles:
   1. Making LLM API calls with streaming
   2. Tool execution (the agent calling bash, browsing web, etc.)
   3. Multi-model failover (if one provider fails, try another)
   4. Subagent spawning (one agent creating another - your "reproduction" 
      system)




---
SECTION 5: THE TOOL SYSTEM

OpenClaw gives agents 60+ built-in tools:
   - Browser - Playwright-based web automation
   - Bash - Shell command execution
   - Canvas - Interactive UI rendering
   - Memory Search - Vector search over past conversations
   - File Operations - Read, Write, Edit Files
   - Session Management - Create/switch between sessions

Tools have a POLICY ENFORCEMENT layer - per-agent and per-session restrictions
on what tools an agent can use. There's also an APPROVAL WORKFLOW where 
dangerous commands require human approval.

CRITICAL FOR YOUR PROJECT: Your agents need tools to make money. OpenClaw's tool
system shows you how to:
   - Define tools with schemas
   - Execute them safely in sandboxes
   - Enforce policies on what agents can/can't do
   - Track tool calls with IDs for auditing






<<<<<>>>>>
SECTION 6: The Plugin & Extension System (`extensions/`, 30+ plugins)

OpenClaw uses a plugin architecture for extensibility:
   - Each plugin lives in `extensions/` as its own workspace package
   - Has a manifest file openclaw.plugin.json defining its ID, channels, and
     config schema
   - Plugins are lazy-loaded and validated with Zod schemas
   - They can provide: new channels, auth providers, tool slots, hooks

Existing plugins include: Discord, Telegram, Slack, Signal...

Why this matters: Your "New Dejima port system" - the interfaces that let agents
   interact with the real world - this IS the plugin system. Each new 
   marketplace, exchange, or service you want agents to access becomes a plugin/
   extension.






<<<<>>>>   
SECTION 7: MEMORY & SEARCH (`src/memory/`)

OpenClaw has a sophisticated long-term memory system:
   - QMD Manager - Hybrid text + vector search with embeddings
   - Multiple embedding backends - Voyage AI, Google Gemini, OpenAI
   - LanceDB - Alternative vector database backend
   - Session-aware indexing - Memories tied to specific conversations
   - Deduplication - Prevents redundant storage

Data is stored as JSONL files at `~/.openclaw/sessions/<agentId>/<sessionId>.jsonl`   

Critical for your project: Your agents need persistent memory to survive across
hibernation cycles. OpenClaw's memory ssytem is exactly the "auto-compaction and
RAG graph" you described in your consciousness loop design.






<<<<<<<>>>>>>>
SECTION 8: THE CHANNEL SYSTEM (`/src/channels`)

Channels are how agents communicate with the outside world:
- BUILT-IN-CHANNELS: Discord (discord.js), Telegram (gramY), Slack..., Signal...
- CHANNEL PLUGINS in `src/channels/plugins/` (40 subdirectories)
- Each plugin registers a channel handler, routing logic, and pairing flow
- Features: allowlist matching, command gating, mention gating, reply perfixes

WHY THIS MATTERS: Your agents need to communicate - send emails, interacrt on 
platforms, contact buisnesses about securiy bugs. The channel system shows how
to abstract multiple communication platforms behind a unified interface.





---
SECTION 9: NATIVE APPS (`apps/`)

- `apps/macos/` (437 Swift files) - macOS menu bar app with voice wake, talk
  mode, canvas rendering
- `apps/ios/` - iOS app with camera, voice, canvas display
- `apps/android` (66 Kotlin files) - Android app with camera, screen recording,
  SMS integration
- `apps/shared/OpenClawKit/` - Shared Swift code for macOS/iOS

                                 <-- ... interesting tbf... and i can kinda see
                                     how this should line up... that is, we can
                                     imagine the case where... idk... 
Why this matters for your 10-hour goal: The Android app codebase 
(`apps/android/`) is directly relevant. It shows you how a Kotlin/Jetpack 
Compose Android app connects to the OpenClaw gateway. This is a reference for 
building the Android app deployment pipeline your agent need.





---
SECTION 10: SKILLS (`skills/`, 50+ bundled skills)

Pre-build tool integrations agents can use:
   - coding-agent - Coding tools
   - github - GitHub integration
   - himalaya - Email client
   - canvas - Canvas control
   - gemini - Google Gemini API
   - gifgrep, food-order, goplaces, healthcheck - Various utilities
   - camsnap - Camera utilities
   - 1password - Password vault access

WHY THIS MATTERS: Each skill is essentially an MCP-like tool pacjage. Your
agents need skills for making money (trading, deploying apps, sending emails).
This shows the pattern for bundling capabilities.   




---
SECTION 11: CONFIGURATION & Security
   - CONFIG: YAML/JSON at `~/.openclaw/config.yml`, validated with Zod schemas
   - CREDENTIALS: Encrypted storage at `~/.openclaw/credentials/`
   - DM PAIRING: Unknown senders need a pairing code (prevent unauthorised
     access)
   - TOOL APPROVALS: dangerous commands require human sign-off
   = SANDBOX: Docker cotnainers for code execution isolation
   - DOCKERFILES: `Dockerfile.sandbox` and `Dockerfile.sandbox-browser` for
     isolated execution.

CRITICAL FOR YOUR SAFETY SANDBOX: OpenClaw already has sandboxing build in. The
   `Dockerfile.sandbox` pattern is exactly what you need for isolating agent
   execution.







---
SECTION 12: BUILD, TEST & DEPLOY

- Build: TypeScript compiled with `tsdawn`, pnpm workspaces
- Test: Vitest with 70% coverage thresholds, unit + E2E + live tests
- Deploy: Docker, Fly.io (`fly.toml`), npm global install
- CI/CD: GitHub Actions with linting, testing, building across all platforms
- Versioning: `vYYYY.M.D` format...



---
WHAT SHOULD YOU FOCUS ON FOR YOUR PROJECT

Given your context, the MOST RELEVANT PARTS of OpenClaw for "New Dejima" are:
   1. `src/agents/` - The agent runtime (how agent think, use tools, manage
      state) - this is your consciousness loop
   2. `src/gateway/` - The control plane (how to coordinate multiple agents) - 
      this is your country administrator
   3. THE TOOL SYSTEM - How to give agents capabilities safely - this is how
      agents make money.
   4. `extensions/` - The plugin system - this is your port/interface system
   5. `src/memory/` - Long-term memory - this is how agents persist across
      hibernation
   6. `Dockerfile.sadnbox` - Sandboxed execution - this is yojur safety layer
   7. `apps/android.` - Android app structure - directly relevant to your
      10-hour goal of agent-deployed Android apps


...                  

   

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡎⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⢦⠀⢀⣤⠶⢖⡒⣢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⠁⡔⢄⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⢩⠀⢡⠏⢀⣠⣼⡆⠀⡩⠤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠎⢠⡆⠈⣦⠀⠀⠀⠀⠀⠀⠀⠀⡎⠌⠀⢈⠔⠡⢦⠛⡏⢶⠁⢀⠑⡀⢀⢔⡆⠙⡱⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⢏⡇⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⡏⣆⠀⠎⠀⠁⠆⠀⡇⠘⣴⣮⣴⠴⡛⢹⠃⠀⢸⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⡡⠈⡇⣘⠀⡇⠀⠀⠀⠀⠀⠀⠀⠇⠙⣷⠇⠀⢰⠀⠀⠁⠀⣻⠏⢀⠎⠀⠈⡄⣠⠟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠱⡄⢰⡜⡐⡇⠀⠀⠀⠀⠀⠀⠀⠰⢡⠈⢃⠀⠸⠰⠀⠀⢀⠇⠀⠀⠀⡀⢀⡿⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⡀⠀⣀⡀⠀⠈⢄⠱⡘⠇⠀⠀⠀⠀⠀⠀⠀⠈⣄⢂⠘⡀⣦⠀⣀⡬⢊⣠⠊⠀⢀⡠⠚⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⡀⡜⢟⠀⠘⠜⢇⢄⠀⠃⢧⡄⠀⠀⠀⠀⠀⠀⠀⠀⠘⢼⡀⡇⡿⢠⣿⣿⣟⣠⣼⡤⠿⠛⠃⠀⠛⠄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⡔⢀⡈⣶⠆⠀⢄⣈⠆⠱⡄⠀⡷⠔⡟⡅⠀⠀⠀⠀⠀⠀⠀⢳⢷⢧⠛⠛⢿⡿⠳⠘⠛⠛⠐⠀⠤⡀⣴⠟⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⣬⠤⠄⠈⡇⠢⠀⠀⠈⠙⠪⣞⡶⠁⠀⠀⣤⠀⠀⠀⠀⠀⠀⣰⠁⢁⠶⠛⠉⠳⢨⡑⣄⡀⠀⣀⠀⡀⡘⣅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⣀⣈⡆⠀⠀⢡⠀⠀⠀⠀⠀⠀⢸⡗⠀⣰⡞⠈⣧⠀⠀⠀⠀⣰⣟⠂⠁⠀⠀⠀⠀⠀⠈⡰⠯⠖⣀⡀⠈⠝⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠈⠳⡌⣩⡁⠑⠚⠛⠢⣀⠑⢤⡐⣮⣷⠾⠋⠀⠀⠸⡟⢿⢍⠍⠉⠄⠉⠐⢦⡀⠀⠀⠀⡞⠀⠀⠑⡄⠀⠒⠂⠒⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠑⠦⠽⠁⠂⠠⠄⠀⡈⠙⢻⢛⢷⠇⠀⠀⠀⢼⡇⠄⠃⠀⠄⠈⢃⠈⠂⠈⠲⡀⠊⠀⢰⠀⠀⢰⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠑⠆⢥⣤⠶⠶⠷⢼⣦⣊⣿⠀⠀⠀⠈⣧⠐⡀⠀⠘⡀⠀⠁⠀⠀⠀⡘⡃⠀⠈⠀⠀⡮⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⣤⡀⢀⣀⡀⠀⠤⣀⠀⠙⢟⠧⠇⠀⠀⠀⣿⡀⠑⠀⠀⠁⠀⡀⢷⡀⠢⣳⢁⠀⡀⠀⣼⡃⢀⢔⡀⢠⠆⣀⣠⡤⠤⢀⡀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠉⠂⠐⢏⣓⠮⠮⡑⠄⢀⠑⢿⡆⠀⢸⠇⠱⣄⠀⠀⠀⠀⠈⠠⢼⠀⢸⣿⢰⣡⣰⣗⠈⠀⡄⣺⣯⡾⡛⠁⠁⠀⠀⠈⠐⠀⢤⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠒⣀⣈⣚⢦⣆⡀⢻⣤⡟⠀⠀⢀⣑⣦⣄⡀⠀⠁⣹⡀⢠⣿⣷⣿⣻⡇⡀⠴⠿⠯⢧⣥⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⣤⠬⢦⣿⢀⡰⢚⠻⠹⡙⠛⣟⣦⡐⢽⣷⣿⣿⣿⣿⢟⣁⣀⠠⠐⠈⠁⡜⠀⠀⠀⠀⠀⠀⠀⢀⡴⠁
⠀⠀⠀⠀⠀⠀⠀⡠⣐⣀⣀⢀⡥⡶⠉⠀⠀⠀⣿⠊⠀⠀⠀⠀⠀⢀⠠⠨⠽⠦⢝⣻⣿⣿⣿⣿⣅⠰⠼⠿⢿⣛⡉⠀⠄⠀⠀⠀⢁⡠⠊⠀⠀
⠀⠀⠀⠀⠀⡀⠜⠀⠀⣀⢶⠡⡘⠀⠀⠀⠀⠀⠇⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⠰⠋⣾⣦⢛⠻⡉⠋⠋⠓⠂⠀⠈⠉⠛⠳⡖⠚⠊⠁⠀⠀⠀⠀
⠀⠀⠀⠀⡐⠉⠀⣠⠊⣁⢂⠔⠁⠀⠀⠀⠀⠀⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⢋⣼⢆⠙⡅⠀⠀⠀⠀⠀⠀⠀⠀⠈⢊⡄⠀⠀⠀⠀⠀⠀
⠀⠀⠀⡠⡂⢔⠙⢕⡸⠇⠀⠀⠀⠀⠀⠀⠀⠀⢸⡦⢀⠀⠀⠀⠀⠀⢀⣠⣲⣌⠵⠚⠁⠀⢀⠈⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⡄⠀⠀⠀⠀⠀
⠀⠠⠖⠁⠂⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⠀⠉⠂⣤⡴⠒⠛⡉⠁⠤⢀⠠⡆⠀⠀⠀⢜⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠃⢠⡞⠉⣤⠊⠁⠈⢠⣴⣭⡜⠀⠀⠀⠀⠀⠉⢢⠀⠀⠀⠀⠀⠀⠀⠀⡜⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⣰⢏⢴⣯⠰⠂⠁⢍⠄⠊⠈⠀⠀⠀⠀⠀⠀⠀⠀⠑⠦⣀⡀⠀⠀⠀⢠⠁⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣥⢾⣕⣂⠈⠉⠩⠭⠉⠁⠖⠂⠤⢀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠁⠈⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⡟⠀⠀⠙⢟⣕⠢⣈⠉⠁⠤⠤⠤⠀⢀⠑⢆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠟⠀⠀⠀⠀⠈⢳⣳⠄⠅⠣⠐⠠⢄⣀⢀⣀⠀⣑⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⡾⠋⠀⠀⠀⠀⠀⠀⠀⠉⠓⠤⣀⡀⠀⠄⠀⠈⢉⣀⣀⣀⣒⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⡾⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣴⠾⠋⠀⠀⠀⠀⠀⠀⠀

OpenClaw Memory System: The Complete Architecture




Layer 1: What Data Gets Stored
   OpenClaw's memory system indexes TWO TYPES OF SOURCES:

   1. MEMORY FILES 
      - `MEMORY./md` or `memory.md` in the agent's workspace root
      - All `*.md` files inside a `memory/` directory
      - Optional extra paths you configure

   2. SESSION TRANSCRIPTS (`source: "sessions"`)   
      - JSONL files at `~/.openclaw/sessions/<agentId>/<sessionId>.jsonl`
      - Each line in the JSONL is a JSON record with type: "message" and a 
        message containing role (user/assistant) and `content`
      - The system parses these, extracts text, redacts sensitive content, and
        converts them into searchable markdown format: `User: <text> / Assistant: <text>`

      This is the key insight: conversations become searchable memory. Every 
      chat session the agent has ever had gets indexed and becomes retrievable.



---
Layer 2: Chunking - How Text Gets Split
   Found in `internal.ts`, the `chunkMarkdown()` function:

   - Takes raw markdown content and splits it into `chunks` based on a token
     budget
   - Each chunk has: `startLine`, `endLine`, `text`, and a SHA-256 `hash`
   - Chunking uses a SLIDING WINDOW WITH OVERLAP - when a chunk fills up, it
     carries over some lines from the end to the start of the next chunk 
     (overlap)  
   - Formula: `maxChars = tokens * 4` (approximating 4 chars per token)  
   - Overlap: `overlapChars = overlap * 4`

   This is similar to what you'd see in LangChain's text spliiters but 
   implemented from scratch. The overlap ensures that if a concept spans across
   a chunk boundary, it still appears in at least one chunk completely.





---
Layer 3: Embeddings - Converting Text to Vectors

Found in `embeddings.ts`. OpenClaw supports 4 EMBEDDING BACKENDS:

1. OpenAI (`embeddings-openai.ts`)
   - Default model: configurable, typically `text-embedding-3-small`
   - Calls the OpenAI embeddings API

2. Voyage AI (`embeddings-voyage.ts`)
   - Specialised embedding provider
   - Has its own batch processing system

3. Google Gemini (`embeddings-gemini.ts`)
   - Uses Gemini's embedding endpoint

4. Local (via `node-llama-cpp`)
   - Default model: `embeddinggemma-300M-Q8_0.gguf` (a 300M param GGUF model)
   - Runs entirely on your machine, no API calls
   - Falls back to this if no API keys are configured


   The provider selection logic in `createEmbeddingProvider()`:
   - If set to "auto": tries local first (if model file exists), then OpenAI,
     Gemini, Voyage in order
   - If a specific provider fails, it falls back to the configured fallback
     provider
   - Every embedding vector gets L2-NORMALISED after generation (divide by 
     magnitude)    

   Each embedding provider exposes two methods:
   - `embedQuery(text)` - embed a single search query
   - `embedBatch(texts[])` - embed multiple texts at once (for indexing)






<<<<<<>>>>>>
LAYER 4: The SQLite Database Schema#

Found in `memory-schema.ts`. Everything is stored in a local SQLite database:

`meta` table - key-value store for index metadata (model name, provider, chunk
size, vector dimensions)

`files` table - tracks which files have been indexed:
   path   TEXT PRIMARY_KEY
   source TEXT ("memory" or "sessions")
   hash   TEXT (SHA-256 of content)
   mtime  INTEGER
   size   INTEGER

`chunks` table - the actual indexed content:
   id TEXT PRIMARY KEY (UUID)   
   path TEXT (∑hich file this came from)
   source TEXT ("memory" or "sessions")
   start_line INTEGER
   end_line INTEGER
   hash TEXT (SHA-256 of chunk text)
   model TEXT (which embedding model)
   text TEXT (the actual chunk content)
   embedding TEXT (JSON array of floats)
   updated_at INTEGER

`embedding_cache` table - caches embeddings to avoid re-computing:
   provider TEXT
   model TEXT
   provider_key TEXT
   hash TEXT
   embedding TEXT
   dims INTEGER
   updated_at INTEGER

`chunk_fts` (FTS5 virtual table) - full-text search index:
   text (searchable)
   id, path, source, model, start_line, end_line (stored but not indexed)

`chunks_vec` (sqlite-vec virtual table) - vector similarity search using the
   `sqlite-vec` extension





---
LAYER 5: HYBRID SEARCH - THE CORE INNOVATION\
   Found in `hybrid.ts` and `manager-search.ts`. This is where it gets #
   interesting. OpenClaw doesn't just do vector search like a basic RAG system.
   It runs TWO SEARCHES IN PARALLEL AND MERGES THEM:

   
   SEARCH PATH 1: VECTOR SEARCH (`searchVector()`)
   - Embeds the query text into a vector using the same embedding model
   - If `sqlite-vec` is available: uses `vec_distance_cosine()` in SQL for
     hardware-accelerated cosine distance
   - If `sqlite-vec` is NOT available; falls back to loading ALL chunks into
     memory and computing cosine similarity in JS (the `cosineSimilarity()`
     function)
   - Returns results sorted by `1 - distance` (higher = more similar)

   SEARCH PATH 2: Keyword/BM25 Search (`searchKeyword()`)    


  Search Path 2: Keyword/BM25 Search (searchKeyword())
  - Uses SQLite FTS5 (full-text search) with BM25 ranking
  - Tokenizes the query: splits into words, wraps each in quotes, joins with AND
  - Example: "machine" AND "learning" AND "model"
  - Converts BM25 rank to a 0-1 score: score = 1 / (1 + rank)

  Merging (mergeHybridResults()):
  - Combines results from both searches by chunk ID
  - If a chunk appears in both results, it gets BOTH scores
  - Final score = vectorWeight * vectorScore + textWeight * textScore
  - Sorts by combined score descending
  - This is called Reciprocal Rank Fusion conceptually, though the implementation is a weighted sum

  This is better than pure vector search because:
  - Vector search finds semantically similar content (meaning-based)
  - Keyword search finds exact matches (when the user uses specific terms)
  - Combining both catches cases where one alone would miss

  ---
  Layer 6: The Manager - Orchestrating Everything

  Found in manager.ts (77KB, the largest file). The MemoryIndexManager class:

  Initialization:
  - Opens/creates the SQLite database
  - Creates the embedding provider
  - Sets up a file watcher (chokidar) on the memory directory to detect changes
  - Sets up a session transcript listener for real-time indexing of new messages
  - Starts an interval timer for periodic re-syncs

  Sync Process (what happens when memory needs updating):
  1. Scans memory files and session files
  2. For each file, computes SHA-256 hash and compares with stored hash
  3. If changed: re-chunks the file, embeds each chunk, stores in database
  4. Cleans up stale entries (files that no longer exist)
  5. Uses batched embedding with configurable concurrency (default: 4 parallel tasks)
  6. Has retry logic: up to 3 attempts with exponential backoff (500ms base, 8000ms max)

  Batch Processing (batch-voyage.ts, batch-openai.ts, batch-gemini.ts):
  - For large indexing jobs, uses provider-specific batch APIs
  - Batches respect token limits (8000 tokens max per batch)
  - Failed batches increment a failure counter; after 2 failures, batching is disabled and falls back to
  sequential

  Session Delta Detection:
  - When a new message arrives in a session, the system detects the delta (new bytes added to the JSONL
  file)
  - Uses a debounce timer (5 seconds) to avoid re-indexing on every single message
  - Only re-indexes the specific session files that changed

  ---
  Layer 7: QMD Manager - The Alternative Backend

  Found in qmd-manager.ts. This is a completely separate backend called QMD (likely "Query Markdown
  Documents"):

  - Instead of doing everything in-process, it shells out to an external qmd CLI binary
  - QMD manages its own collections, indexing, and embedding
  - The QmdMemoryManager class wraps QMD CLI commands: qmd collection add, qmd update, qmd embed, qmd query
  - Stores its index in ~/.openclaw/state/agents/<agentId>/qmd/
  - Has its own XDG-compliant directory structure (config home, cache home)
  - Runs qmd update on boot and at configurable intervals (default: 5 minutes)
  - Runs qmd embed separately with a longer interval (default: 60 minutes)
  - Has scope-based access control: can restrict memory search based on channel type (direct messages only
  by default, groups denied)

  Session Export for QMD:
  - Exports JSONL session transcripts as .md files into a dedicated directory
  - QMD indexes these markdown files as a "sessions" collection
  - Has retention policy support (delete sessions older than X days)

  ---
  Layer 8: The Fallback System

  Found in search-manager.ts. The FallbackMemoryManager class:

  - Wraps the primary backend (QMD) with automatic fallback to the builtin backend
  - If QMD fails during a search, it permanently switches to the builtin MemoryIndexManager
  - Evicts the failed manager from cache so the next request gets a fresh instance
  - This ensures memory search never completely breaks even if the external QMD binary has issues

  ---
  How This Relates to Your Project

  For your agents' consciousness loop, here's what you'd adapt:

  1. Session-as-memory: Every action your agent takes gets stored as a JSONL transcript. When the agent
  wakes up from hibernation, it can search its own history to remember what it was doing. This IS the
  "auto-compaction and RAG graph" from your design.
  2. The hybrid search is crucial: Your agents need to find both semantically related memories ("what was I
   working on related to making money?") AND exact matches ("what was the Stripe API key I used?"). Pure
  vector search would miss the second case.
  3. The local embedding option matters for cost: If your agents are trying to be cost-efficient, using the
   300M GGUF model locally for embeddings instead of paying for OpenAI embedding API calls could save
  significant money at scale.
  4. The delta detection pattern: You don't re-embed everything every time. You only re-embed what changed.
   This is critical for agents that run for weeks - you can't afford to re-process all memory on every wake
   cycle.
  5. The scope system: QMD's scope-based access control (deny memory search in group chats, allow in direct
   messages) maps directly to your agent permission model - different contexts get different memory access
  levels.

  Ready for the next section whenever you are.


In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢞⡗⠪⡷⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣄⠀⠀⠈⡝⠃⡃⠀⠀⢀⣐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣋⢰⡆⠄⣰⡾⡒⣲⡥⣻⣙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠟⠼⠊⠘⢱⢛⠏⠈⠛⠹⣝⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⢀⡜⢧⡧⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣫⢭⡻⣙⠣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⡀⡞⡍⣥⠟⣦⡤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⢄⡶⢷⢸⠧⢙⠻⡰⡟⡿⠎⣷⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⢤⢄⢀⡀⣸⢾⢳⠂⣠⡓⣽⣦⣻⠖⡗⣤⠗⡷⣞⣻⣢⣤⣄⠄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⢀⡤⣦⡴⣔⠥⠟⠓⠙⠘⠱⢴⢮⢩⡼⠨⢙⡶⡄⣬⣸⡡⠃⣌⡫⣬⣗⢹⣦⣱⡘⠋⢙⠿⡸⡽⣲⡠⢤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢀⣠⣠⣏⠺⠛⠘⠀⠀⠀⠀⠀⠀⢴⣤⣾⢯⢮⡬⢧⣛⣾⢁⡹⢹⡥⣸⡻⢧⢫⣋⣬⣷⣩⢷⠀⠀⠀⠈⠀⠋⠍⡶⢏⣓⣢⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢴⢟⣱⣴⠻⠀⠀⠀⠀⠀⠀⠀⠀⠀⡱⣟⡖⣽⣶⢆⣶⣦⣡⢨⣄⣠⡝⠃⣦⣆⣠⣫⣆⢸⣿⣾⡁⠀⠀⠀⠀⠀⠀⠈⠑⡞⡧⣚⢷⣆⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠐⢗⢸⡳⢿⣁⣇⡀⠀⠀⠀⠀⠀⠀⠀⠑⢺⡇⡟⣙⣔⣿⣻⡍⠿⣬⣗⣇⠟⣌⣟⣂⡟⢿⡞⡿⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⣞⡽⠇⢽⠆⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠳⢟⡼⣽⣻⣱⣦⣄⣀⠀⠀⠀⠀⠀⠈⠿⡾⡋⣟⢷⣻⣟⡿⣳⣟⣝⣯⡳⣼⣧⡻⡿⣽⠃⠀⠀⠀⠀⠀⠀⢀⢀⡴⣾⣿⡮⣺⣡⠟⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠘⠚⠵⣆⣿⡴⠾⢴⣦⢄⣤⣤⣼⣞⣈⡉⠚⢙⣃⠒⠛⠃⢚⡛⠻⣨⣼⠾⢇⣅⣀⣀⡤⣄⣔⣶⣎⡳⡣⣽⡣⠷⠛⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠛⠻⠮⠯⢽⣇⣚⣿⣯⠿⡹⣿⣿⡯⢻⣮⣻⢏⡻⣍⣸⣱⣭⣍⣭⡷⠯⠵⠛⠚⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠻⠛⠛⠿⠿⢒⠻⠟⠉⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

### temp

... Pi is a codename for an external AI agent SDK called pi-mono, created by...

It provides:
   - `pi-ai` - LLM abstractions (model interfaces, streaming, provider APIs)
   - `pi-agent-code` - agent execution loop and tool framework
   - `pi-coding-agent` - High-level SDK with `createAgentSession()`, auth, model
     registry, built-in coding tools
   - `pi-tui` - Terminal UI components


---
"Pi Embedded Runner" is OpenClaw's integration layer that EMBEDS THE PI-MONO
AGENT RUNTIME DIRECTLY into the OpenClaw gateway process (as opposed to running
it as a separate subprocess or via RPC).

It lives in `/src/agents/pi-embedded-runner/` and handles:
   - SESSION MANAGEMENT - JSONL-based persistence with branching and compaction
   - MODEL RESOLUTION - Integrates with pi's `ModelRegistry` and `AuthStorage`
   - TOOL ADAPTATION - Maps OpenClaw's tools into pi's agent loop format
   - SYSTEM PROMPT BUILDING - Dynamically constructs prompts per cahnnel/context
   - EVENT STREAMING - Streams agent events (messages, tool calls, thinking) 
     back to clients
   - FAILOVER - Context overflow detection, auth failure handling, model 
     switching


The word "embedded" means the pi agent runs IN-PROCESS - OpenClaw directly 
imports and instantiates pi's `AgentSession`, giving it full control over the
lifecycle, tool injection, prompt customisation, and session persistence.







### definition

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⡀⠀⠀⠀⢀⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⢀⣀⣀⣀⣀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠻⣄⠀⣯⠳⢷⠾⢻⣷⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢸⠁⠀⠀⢸⠀⠀⡀⢵⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⡤⢄⣀⠀⠀⠙⣶⣿⠆⠀⠀⠘⠿⣦⠀⠀⠀⠀⠀
⠀⠀⣠⠴⠒⢲⡦⣄⣰⠀⢰⡄⢸⢃⡄⡿⣸⢀⣠⠤⠦⠤⠤⣀⠀⠀⠀⠀⠀⠀⠀⣼⠋⠀⠀⠈⢦⠀⠀⢾⣴⣶⡆⢀⣶⠶⠚⠃⠀⠀⠀⠀
⠀⢰⣧⣤⢦⠀⠀⠆⢉⣦⡘⢟⠣⢿⠃⡇⡿⢁⡈⣀⣀⠀⠀⠈⢦⠀⠀⠀⠀⠀⠘⣿⣦⢠⣴⣦⠜⡆⠀⠀⢹⡈⢻⣾⠃⠀⠀⠀⠀⠀⠀⠀
⢠⠯⠀⠈⠉⠀⠎⠑⢶⣾⣷⡞⢆⣟⡼⣠⠩⡯⠛⠅⠀⣀⡤⠴⠦⠇⠀⠀⠀⠀⠀⠙⠋⠀⠛⠧⠀⡇⠀⣀⣈⣇⣸⠡⣴⣴⠀⠀⠀⠀⠀⠀
⠸⠿⠹⠿⠒⣢⣄⣠⠄⣈⡿⣷⣽⡿⣷⣟⣞⠔⠳⡒⠾⠣⢄⡀⠀⣀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⣖⡯⠭⠖⢲⡧⣏⠑⢮⡀⠀⠀⠀⠀⠀⠀
⠀⠀⣠⠞⠉⠈⠁⢼⡭⠽⣟⣽⣿⣷⣛⠻⠏⡤⡦⢤⣠⠀⠀⡙⢿⠸⢷⡀⠀⠀⠀⠀⠀⣴⡿⠋⢹⠁⠀⠀⢸⣸⠉⠙⡄⠘⢦⢀⡀⠀⠀⠀
⠀⢰⢁⡔⠢⠖⠛⠆⠸⠌⠉⣽⣿⣿⡯⠤⡨⠬⠥⣀⣀⠀⠀⢨⠃⠀⢸⡟⣆⠀⠀⢠⡾⠁⠀⢀⠇⠀⣄⡄⣼⠇⠰⡤⣼⣤⣼⠏⣧⠀⠀⠀
⠀⣄⣎⡞⢀⡤⠟⣀⣬⡤⡞⢗⠁⢻⣰⣪⢄⡀⠀⠉⠀⠙⠢⡏⠀⠀⠀⠀⢸⠀⢠⡟⠀⠀⠀⡜⠀⠈⠟⣹⠏⠀⠀⣽⡄⠉⠁⠀⠿⣧⣀⠀
⠀⠹⡟⣰⣿⠒⠉⠁⠀⠀⡏⠈⢀⢸⡅⠀⠉⠾⡀⠀⠀⠀⠀⢱⠀⣰⠀⣀⠗⠒⠉⠉⠉⠉⠻⡁⠀⠀⣰⠋⠀⠀⢘⡿⠃⠀⠀⠀⣤⠔⠛⠃
⠀⠀⠱⠿⠃⠀⠀⠀⠀⠀⢧⠀⠘⠸⠙⣄⠀⠀⠀⣤⡠⠀⠀⡾⣲⠛⡸⢛⡀⢀⡀⣀⡠⠔⠒⢱⡄⡴⠃⠀⠀⠀⠚⠒⣻⠿⣦⣰⠇⡀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢶⡄⠀⠀⠈⢦⣀⡀⠈⣻⢣⢲⣱⢁⠴⣡⢔⣮⠟⠊⠁⠀⠀⠀⣸⠟⠁⠀⠀⠀⢀⣠⡾⠁⠀⠈⠹⢰⠁⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢠⡀⠀⠀⡈⢪⡀⢄⣧⠶⢛⡢⠆⣶⣯⣾⣷⣷⣿⡾⢇⣤⠆⠀⠀⢀⣤⠞⠃⠀⣴⣒⣺⠿⠟⠋⠀⠀⠀⠀⠀⡞⣰⣤⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠳⣔⢺⣷⡋⡽⠋⢁⣠⡼⢷⣾⣿⣿⣿⡿⣽⡝⡉⣑⡢⠽⠔⠚⢿⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⣷⠄⡼⠀⠚⠉⠀
⠀⠀⠀⠀⠀⠀⠀⠶⠗⠈⣑⡶⠋⢀⡀⠈⠀⠀⠄⢢⡋⠁⠹⡧⠀⠉⢻⡙⠓⠢⠀⠀⠀⠹⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢈⡜⠁⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢠⢾⣟⣧⣷⣋⣁⣀⣀⠤⠔⠚⡟⠐⠀⢀⡇⠈⠀⠰⣇⡀⠀⠀⠀⢄⠀⢱⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⠏⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢸⠈⠉⣋⢀⣰⡴⠈⢆⠀⠀⠀⡇⠀⠀⢠⠇⠀⠇⢠⠗⠋⠒⠦⣄⠈⣇⡞⠀⠀⠀⠀⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠑⠤⠿⠁⠛⡩⠚⠉⡟⠲⡄⡇⠀⠀⠀⠀⣾⢠⠎⣀⡤⠴⠾⠭⠶⢍⣉⡉⠓⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣤⣇⣀⠀⢸⡠⠼⣇⠀⡴⠓⢦⣿⣯⢮⠥⠄⠠⠤⣄⠀⠀⠀⠈⠙⠲⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠾⡇⣱⣧⡴⠙⡧⣾⠟⠻⣶⣶⣯⣿⣵⣫⠀⠀⠀⠀⠀⠑⢦⠀⠀⠀⠀⠈⢻⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠟⣉⡡⡟⠢⣧⡜⡦⠜⣿⡯⣬⣽⣱⠟⡠⡄⠀⠀⠀⠀⠈⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠃⠻⣯⡍⠁⢉⣳⠟⢿⠇⠉⢳⡀⢿⡇⠀⠀⠀⠀⢠⠃⠀⢀⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣹⣥⣶⣜⡟⡔⠙⠑⠢⡀⣑⠤⠹⢦⠤⠤⢒⣵⣿⣽⡿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠾⠿⣎⢿⠀⠀⠀⢠⣏⠀⠀⠀⠀⢱⡐⠺⠿⠿⠟⠁⠀⠶⠶⢤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣴⢧⣀⡈⠻⡦⣀⡠⣷⡿⠇⠀⡀⠀⢈⠼⣑⣒⡤⠤⠄⠀⠀⠀⠀⢹⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣢⣠⡎⡴⠎⠋⠽⢿⣿⣿⣽⣿⣵⢆⡼⢤⣄⠀⠀⠀⠀⢠⠞⣭⡝⣿⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠈⠃⢣⡀⠀⠀⠀⢺⣿⣿⣿⣾⠉⠈⠁⠀⢳⠀⠀⠀⠻⠀⢈⡵⢻⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠤⠒⡂⢤⣠⠚⠙⠉⢻⠀⠀⠀⠀⣽⡟⢵⣈⣀⣤⡶⠶⠊⠀⠠⢼⣆⡴⣋⡴⣿⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢻⣆⠀⠈⠫⡇⠀⠀⠀⠈⠓⠤⢠⡔⢫⡘⠄⠀⠀⢱⢀⣀⠀⢀⣴⢠⣾⠿⠧⢤⡀⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡠⡶⠉⠩⠵⢶⣿⣰⡀⠀⠀⣀⠀⢨⣿⡦⡉⠒⠒⣒⣯⢘⣏⠛⠋⣸⣋⠁⠀⠀⠀⢹⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⡞⡉⠀⠀⢀⡤⣶⣻⣿⣷⣶⣿⣶⣶⣅⣿⣿⣌⢦⣼⣠⣤⣸⣋⣀⠀⡤⠭⠓⠀⠐⠒⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⢀⣶⣳⠀⠀⠀⠀⠀⠙⠪⠬⣵⠅⠀⠤⡞⣛⣿⣿⠏⢱⡉⠙⢝⣖⠿⣿⡝⢮⣭⡤⣴⠟⣾⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣰⡿⣻⣛⢳⡀⠀⢀⣀⣤⣤⣄⡇⠀⠀⢀⡀⠁⠹⡼⠀⠀⢸⢀⢀⣼⠀⠀⢙⣀⣤⢚⣯⣀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢾⣿⣿⣿⣿⠿⠦⢄⣘⣟⣿⣿⡷⣄⠴⠋⠀⢀⡴⣇⠀⠀⢸⠉⠁⢈⢷⠭⣉⠘⠟⣺⠉⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠘⠽⠛⠁⣠⢶⡶⠋⠀⠉⠉⢀⠙⠢⢶⣒⡽⠹⣮⣵⡒⠋⠀⠀⠸⣿⢏⢗⢷⠀⢸⡠⣶⠂⠘⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠈⠉⠀⠀⠀⣠⣦⠟⢀⣾⣿⣿⠇⣇⠻⣝⣻⣆⡀⠀⠀⠹⣼⡧⣄⡁⠀⠛⠦⠴⠚⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢀⠀⢠⡏⠀⢘⢟⡫⠥⣴⡿⡿⣢⠤⣄⠉⠀⠀⠀⠀⠉⠙⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠐⢺⠯⡾⠳⣀⠔⡞⣾⡿⢿⣝⢼⠋⢀⣧⠤⢶⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⣀⣀⠀⣀⡄⢰⠇⠀⠀⠀⣟⣚⣷⣀⡰⣷⣠⠟⣃⢀⡞⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠘⣏⠛⢹⡇⢸⢠⢀⠀⢀⣟⣁⣀⡩⣾⣾⣿⡿⢯⣥⠭⢭⡿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠾⣥⡄⣰⠾⢻⡿⠟⣔⢿⣿⠋⠈⢊⡿⢻⡿⣿⡀⠙⡽⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠙⠏⠀⠘⣇⢰⣿⠿⠿⠠⠶⢻⠂⢸⡧⠈⢷⠢⢤⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣽⣎⣡⡄⠀⠀⠀⠸⣆⡜⠳⣧⡼⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⡈⢹⣮⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⢠⣿⡳⣦⣀⠀⠀⠀⣀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⢿⣶⡈⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢀⡴⠀⠀⠀⣠⢸⡏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢸⡀⠀⢀⣸⡏⠻⢷⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠳⣤⣉⣳⠄⣀⢻⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⠚⠉⠉⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

- I believe it would be useful to ask Claude to explain to me the key functionalities
  of each of the file's functionalities... as in... explain the files that
  are top 5 or top 10 important for each section that's related to my current
  project... and then i will just go through them
     - tbh we can also share in the plans too... of how we want to be the
       super-architect... and i guess it is hence from here, that it would be
       incredibly important for us to know what files to change what, and where
     - come to think of it... would it be useful to fill up an additional 
       sub-directory which consist of a skeleton repository... like a tree
       directory... that's a direct one-to-one copy of the original openclaw repo
       but instead of having modified code... we first only have and is filled with
       md files that plans on what we want to implement... and why are these
       certain files useful, etc.
          - in fact we need to start doing this for android project already...
          
            

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⣶⣶⣶⣶⣶⣶⣶⣶⣶⣶⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢰⣶⡎⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⢱⣶⣶⣶⣶⣶⣶⡆⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢣⣤⣤⣤⣤⣤⣤⣤⣤⡄⠀⠈⠛⠛⠛⠛⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⠛⠛⠛⠛⠛⠛⠛⢣⣤⡄⠀⠀⠀⠀⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⡸⠿⢇⣀⣀⠀⠀⠻⠿⢃⣀⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⠿⠿⢇⣀⡸⠿⠿⣀⣀⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⣿⣿⡇⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⡇⠀⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⢰⣶⡎⠉⠱⣶⣶⠉⠉⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⣶⣶⡌⠉⢱⣶⡆⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⣴⣶⡎⠉⠁
⠀⠀⠀⣤⣤⠛⠛⢡⣤⡜⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠛⠃⠀⠀
⢀⣤⣤⠛⠛⣤⣤⡘⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢸⣿⣿⣀⣀⠿⠿⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠸⠿⠿⠿⠿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

# PLANS

We can instead try and be the super-architect... by asking Opus WHICH DIRECTORIES
AND IN WHICH EXISTING FILES inside the openclaw/openclaw repository... that we
should introduce changes
   - and for each file, add what features
   - and where should each feature of dejima be added





- look up $BUNKER, and ask Claude Code to analyse it... and ask it what portion
  of $Bunker can we use
     "/Users/administrator/Imperial-College-London/Projects/2026/2026-02 February/Project Altiera/resources/moltbunker-sdk"



- look up Grok 4.20 SITUATIONAL-AWARENESS
   - we want to analyse what methods were given to achieve such incredible rates
     of returns... if this system can be continuously demonstrated to be 
     profitable... then this system is already viable...   




- The integration of the `pi-mono` repository... definitely needs to be heavily
  considered!!     



- Additional system-checkers on top of `playwright` for validation of android
  apps' publishment... should definitely be paramount
        - check the `apps/android/` directory and `apps/ios` directory

In [ ]:
⠀⠀⠀⠀⠀⠀⡶⠒⠒⠒⠒⢒⣦⡀⠀⠀⠀⠀⣶⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠒⣶⠀⢀⡖⠒⠒⠒⠒⢲⣶⡶⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠓⠒⠒⢲⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠲⣶⣶⠒⠚⠛⠚⠚⠳⢖⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠚⠚⠛⠓⠒⡖⠒⠒⢶⠀⠀
⠀⠀⠀⠀⠀⣸⠁⠀⠀⠀⠀⣼⣿⣷⡄⢀⣀⣰⡃⠀⠀⠀⠀⢰⣿⡇⠀⠀⠀⠀⢰⣿⡇⠀⠀⠀⠀⢀⣿⣇⢸⣇⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⣿⠀⠀⠀⣀⣀⠀⠀⣀⣀⡀⠀⠀⢿⣿⡀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠀⠀⢹⣿⡆⠀⠀⠀⠀⠀⠀⠀⠀⣷⠀⠀⠘⡆⠀
⠀⠀⠀⠀⢠⠇⠀⠀⠀⠀⠀⠀⢸⣿⣿⣾⠃⠀⠀⠀⠀⠀⠀⣾⣿⠀⠀⠀⠀⠀⣸⣿⠁⠀⠀⠀⠀⠀⠀⣼⣿⠁⠀⠀⠀⠀⣹⣿⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣷⣀⣀⡄⠀⠀⣿⣿⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠿⡿⣇⣀⡽⠿⠇⠀⠀⢸⣿⡇⠀⠀⠀⠀⠘⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡆⠀⠀⠀⠀⠸⣿⣿⡿⠁⠀
⠀⠀⠀⠀⡞⠀⠀⠀⠀⠀⠀⠀⠛⠛⠛⠛⠒⠒⠒⠒⠓⠒⢲⣿⡇⠀⠀⠀⠀⢀⣿⡿⠀⠀⠀⠀⠀⠀⠀⠛⠛⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⠘⠛⠛⢻⠛⠛⠛⢻⣿⡇⠀⠀⠀⠀⠲⣿⣿⣿⣿⣿⣿⣿⣿⣿⠀⠀⠈⠁⠀⠛⠛⠀⠀⠀⢠⣤⣼⣿⡇⠀⠀⠀⢹⣿⣿⣿⣶⠀⠀⠀⠈⣿⣷⠀⠀⠀⠀⠀⠛⠛⠛⠛⡛⠛⠛⢻⣿⣿⣿⣿⣷⠀⠀⠀⠀⠀⢻⡋⠀⠀⠀
⠀⠀⠀⣸⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⣿⠁⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⢠⣶⡞⠀⠀⠀⠀⠀⠀⠀⣿⡟⠀⠀⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⢹⣿⡇⠀⠀⠀⠘⠟⠀⠉⠛⠂⠀⠀⠀⢻⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠈⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠘⣇⠀⠀⠀
⠀⠀⢠⡏⠀⠀⠀⠀⢀⣿⡿⠀⠀⠀⣸⣿⠇⠀⠀⠀⠀⢠⣿⡏⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⣸⣿⡇⠀⠀⠀⠀⡀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣿⠇⠀⠀⠀⠀⢠⡇⠀⠀⠀⠀⣿⡏⣿⣿⠀⠀⢀⡀⠀⣸⣿⡇⠀⠀⠀⠀⠈⣿⣷⠀⠀⠀⠀⠀⣀⣀⠀⠀⠀⠀⠀⠸⣿⡇⠀⠀⠀⠀⠈⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣀⠀⠀⠀⠀⢹⡄⠀⠀
⠀⠀⡾⠀⠀⠀⠀⠠⣾⣿⣅⣀⣀⣠⣿⡿⠀⠀⠀⠀⠀⣼⣿⠃⠀⠀⠀⠀⢰⣿⡏⠀⠀⠀⠀⠈⣿⣿⣿⣿⠀⠀⠀⡈⠀⢸⣿⡇⠀⠀⠀⠀⠀⠿⠿⠿⠿⠿⢿⠿⣿⣿⠀⠀⠀⠀⠀⠘⠷⠶⠶⠶⠶⠿⠿⣿⣿⠀⢸⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⢸⣿⡗⠒⠀⠠⠄⠤⣿⣿⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⠟⣿⣿⣿⣿⣿⣿⡏⠁⠀⠀⠀⠀⢷⠀⠀
⠀⢸⠇⠀⠐⠀⠀⢠⣿⣿⣿⣿⣿⣿⣿⠃⠀⠀⠀⠀⢀⣿⡟⠀⠀⠀⠀⠀⣸⣿⠃⠀⠀⠀⠀⢀⣿⣿⣿⡏⠀⠀⠀⠇⠀⣾⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⡔⠁⠁⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢂⠀⠀⢿⣿⡀⡸⠀⠀⠀⠘⣿⣧⠀⠀⠀⠀⠀⢻⣿⡄⠀⠀⠀⠀⠈⣿⣧⠀⠀⠀⠀⠀⢹⣿⡆⠀⠀⠀⠀⠈⣿⣿⣿⣿⣿⡆⢹⣿⣿⣿⣿⣿⣿⡀⠀⠀⠀⠀⠘⡆⠀
⠀⢿⣦⣤⣤⣤⣴⣾⣿⣿⣿⣿⣿⣿⣿⣦⣤⣤⣤⣴⣾⣿⣷⣤⡾⣦⣤⣴⣿⣿⣦⣴⣶⣤⣤⣾⣿⣿⣿⣷⣤⣦⣶⣶⣴⣿⣿⣦⣤⢾⣦⣶⣦⣶⣶⣶⣷⣶⣶⣶⣿⣿⣦⣤⣴⣰⣿⣶⣤⣤⣤⣴⣾⣤⣴⣾⣿⣷⣷⣴⣶⣦⣶⣿⣿⣦⣴⢢⣤⣤⣾⣿⣷⣤⣴⣶⣤⣴⣿⣿⣶⣶⣶⣶⣴⣾⣿⣷⣶⣤⣶⣦⣶⣿⠋⢿⣿⣿⣷⠀⣿⣿⣿⣿⣿⣿⣷⣶⣤⣤⣴⣴⠟⠀
⠀⠀⣹⢿⣿⣿⣿⣛⢻⣿⣿⣿⣿⡟⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣇⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡛⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣛⣛⣿⣿⣿⡟⠁⠀⠸⣿⣿⣿⡆⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣇⠀⠀
⠀⢠⡟⠀⠙⢿⣿⣿⣿⣿⣿⣿⣿⠁⣾⣿⠉⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠁⢻⣿⣶⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠏⠀⠀⠀⠀⢻⣿⣿⣧⠀⣿⣿⣿⣿⣿⣿⣿⡿⠋⠀⢻⡀⠀
⠀⣼⠁⠀⠀⠀⠈⠉⠉⠉⠉⠉⠁⢠⣿⡏⠀⠀⠈⠉⠉⠉⠉⠉⠉⠈⠉⣹⣿⠉⠉⠉⠈⠉⠉⠉⠉⠉⠩⠀⢸⣿⡏⠉⠉⠉⠉⠁⠉⠉⠉⠉⠉⠉⣽⣿⠋⠉⠉⠉⠉⣿⡏⠉⠉⠉⠉⠉⡏⠀⠀⠀⣿⣿⡏⠉⠉⠉⠉⢹⣿⣿⣿⣿⣿⣿⡏⠉⠉⠁⠉⢹⣿⡍⠉⠉⠁⠉⠉⠉⠉⠉⠉⠁⠉⣿⣏⠉⠉⠉⠁⠀⠀⠀⠀⠀⠘⣿⣿⣿⡄⠈⠉⠉⠉⠉⠉⠉⠀⠀⠀⠈⣧⠀
⢰⡏⠀⠀⢀⣄⢀⡠⠤⠠⠤⠤⠤⣼⣿⠁⠀⠀⠀⠀⠠⠤⠤⠤⠤⠤⣴⣿⡏⢀⣀⠤⠤⠤⠤⣤⡤⢤⣆⡠⣼⣿⡇⠀⠀⠀⠀⠤⢤⠤⠤⠠⠤⠤⣿⣿⠀⠀⡄⠀⢤⣿⡏⠀⠀⠀⠀⢀⡇⠀⠀⠀⣿⣿⣿⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣇⠀⢠⣄⡠⢼⣿⡇⢀⠀⠀⠤⠤⠤⠤⠤⠠⠤⠤⣿⣿⣀⣀⢤⣄⣀⡠⠤⠤⠤⠤⢿⣿⣿⣧⡀⠀⠤⠤⠤⠄⠀⢀⠀⠀⠀⢹⡄
⠘⠷⣄⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠃⠀⠈⢿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⠀⠀⠁⠀⠀⣿⣇⠀⠀⠀⠀⠀⡇⠀⠀⠀⣿⣿⡇⠀⠀⠀⠀⢠⣿⣿⣿⣿⣿⣿⠃⠀⠀⠁⠀⣸⡿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠏⠀⠀⠀⠃⠀⠀⠀⠀⠀⠠⢾⣿⠟⠁⠀⠀⠀⠀⠀⠀⠀⠜⠀⠀⣠⠞⠃
⠀⠀⠈⠳⣤⡀⠀⠀⠀⠀⢤⣤⣤⣤⣤⣬⣿⣦⣤⣤⣤⡀⠀⠀⠀⠀⢤⣤⣼⣿⣆⠀⠀⠀⠀⠠⣤⡀⠀⠀⠀⠈⢿⡆⠀⠀⠀⠀⢠⣤⠀⠀⠀⠀⠈⣿⡇⠀⠀⠀⠀⢸⣿⠀⠀⠀⢀⣠⡇⠀⠀⠀⣿⣿⠁⠀⠀⠀⠀⠚⢻⣿⣿⡿⠛⠋⠀⠀⠀⠀⣰⡿⠁⠀⠀⠀⢀⣤⠆⠀⠀⠀⠀⣰⡿⠁⠀⠀⠀⢀⣤⠆⠀⠀⠀⠀⣴⠟⠁⠀⠀⠀⣠⣤⣤⣤⣤⣤⣤⡴⠞⠁⠀⠀
⠀⠀⠀⠀⠈⠙⢦⣀⠀⠀⠀⠙⠿⠿⠿⠿⠿⠿⢿⣿⣿⣷⣄⠀⠀⠀⠈⠻⣿⣿⣿⣿⡉⠀⠀⠀⠙⣷⡄⠀⠀⠀⠈⣿⡄⠀⠀⠀⠀⠻⠧⠀⠀⠀⣀⣻⣷⠀⠀⠀⠀⠸⠿⠀⠀⠀⠀⠀⡇⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⠉⠿⠿⠿⠇⠀⠀⠀⠀⠀⢠⣿⠁⠀⠀⠀⢤⣿⠏⠀⠀⠀⢀⣾⠟⠀⠀⠀⢀⣴⡟⠁⠀⠀⢀⣴⡾⠋⠀⠀⠈⠩⠿⠿⠿⠿⠿⣿⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠙⠷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠙⢿⣿⣿⣷⡄⠀⠀⠀⠘⣿⡄⠀⠀⠀⠘⣿⡄⠀⠀⠀⠀⢀⡀⠀⠀⠙⠻⣿⣦⠤⢴⡄⠀⠀⠀⢠⣤⠤⠼⠇⠀⠀⠀⣿⡏⠀⠀⠀⠀⠀⡀⠀⠀⠀⣀⠀⠀⠀⠀⢠⣿⠃⠀⠀⠀⢠⣿⠃⠀⠀⠀⣠⣿⠃⠀⠀⠀⣠⡾⠋⠀⠀⠀⣠⡾⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠞⠋⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠛⣿⣿⣿⣿⣷⣄⠀⠀⠀⠙⢿⣿⣿⣧⡀⠀⠀⠈⠻⣿⣿⣿⣆⠀⠀⠀⠈⢿⣄⠀⠀⠀⠘⣿⡀⠀⠀⢀⣹⣷⠀⠀⠀⠀⢻⣿⠀⠸⡇⠀⠀⠀⠀⣿⠀⠀⠀⠀⠀⠀⣿⡇⠀⠀⠀⢰⣿⠃⠀⢀⣾⡏⠀⠀⠀⢠⣿⠃⠀⠀⠀⣰⡿⠁⠀⠀⠀⣼⡟⠁⠀⠀⢠⣾⠏⠀⠀⠀⣠⡾⠋⠀⠀⢀⣠⣾⣿⣿⣿⣟⠛⠛⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢧⣀⠀⠀⠀⠁⠀⠀⠀⠉⠛⣿⡏⠙⢦⡀⠀⠀⠙⢿⡇⠘⢷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣷⡀⠀⠈⠉⢿⣧⠀⠀⠀⠸⡏⠀⠀⣿⠀⠀⡀⠀⣿⠀⠀⠀⠀⠀⠀⣿⠁⠀⠀⠀⣼⠛⠛⠛⢻⡿⠀⠀⠀⢀⣿⠃⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠋⠉⠉⠀⠀⠀⠀⠀⣠⣤⡾⠋⠀⠀⠀⡀⠀⠀⠀⠀⣠⡴⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠁⠀⠈⠛⠛⠛⠛⠛⠃⠀⠀⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠃⠀⠀⠙⠛⠛⠛⠛⠃⠀⠀⠀⠀⠀⠀⠙⠛⠛⠛⠛⠋⠀⠀⠀⠘⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠁⠘⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⢄⣠⠤⠤⣄⡐⣢⠥⠤⢤⣀⡖⠉⠉⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡟⢠⠞⢳⡄⢻⢁⡔⠻⣦⠙⣗⣦⣴⣧⡤⣤⡀⠀⠀⢠⡤⠤⠤⣄⣀⣀⣀⡴⣤⡴⣄⣦⢦⠠⡀⠀⣄⡴⣄⡀⠀⠀⠀⠀⢀⠔⣤⠦⠤⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡊⠀⠙⣄⡤⠤⠤⡀⡀⠀⠀⠀⠀⡀⢠⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢧⠘⠧⡽⢏⣼⡈⠳⡬⠟⣠⢯⠃⢻⠏⠶⠞⢹⠀⠀⢘⣶⡆⣶⣾⢟⣛⡻⡇⢸⡇⣿⡋⠘⣳⡖⣛⣾⡇⣿⡴⣒⢒⢦⡀⠈⢸⠉⡖⠒⢻⡖⡒⡲⡖⢒⣖⢦⡶⢒⣒⢦⢖⣒⢒⣷⡦⠒⢿⡀⠒⠲⡿⠚⢒⢶⠒⢖⠒⠿⢺⠗⠓⡲⣤⠖⣒⢒⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢲⡖⢋⠭⢉⠲⡶⡚⠏⠂⠀⣼⠀⣾⡆⢸⠀⠀⠀⢿⡇⡟⢻⡠⠬⠤⣇⠸⣇⢿⣿⡘⢟⡣⠤⠀⣇⢿⣄⠤⠥⢼⡇⠐⢸⡀⠿⢇⣸⡡⠤⠄⡇⢠⣼⠸⣀⠤⠤⢼⠦⠤⢝⡯⠇⠈⣞⠹⠿⢀⣇⠤⠤⢼⠀⣾⣷⠆⢸⡀⠤⠤⡾⠦⠤⢙⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡀⢷⠈⢸⠆⣹⠇⠀⠀⠀⠈⠛⠁⠙⠋⠀⠀⠀⠀⠙⠁⠈⠉⠚⠚⠉⠛⠉⠋⠁⠑⠉⠑⠛⠛⠉⠛⠉⠛⠓⠋⠁⠀⠁⠉⠒⠊⠁⠙⠛⠚⠛⠁⠈⠊⠉⠛⠒⠛⠓⠒⠉⠀⠀⠀⠈⠉⠒⠉⠈⠑⠓⠊⠙⠁⠃⠈⠋⠉⠒⠒⠋⠓⠒⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣤⣉⣡⠴⡣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

- WebSockets are a computer communication protocol providing full-duplex,
  bidirectional communication channels over a single, long-lived TCP connection.
  Unlike HTTP's request-response model, WebSockets allow servers to push data
  to clients insantly, making them ideal for low-latency, real-time applications
  like chat, gaming and live financial feeds.

- RPC (Remote Procedure Call) is a client-server networking protocol that allows
  a program to execute code on a remote server as if it were a local subroutine,
  withut needing to understand network details. It uses stubs, serialisation
  (marshalling), and protocols like TCP or HTTP to facilitate communication,
  commonly used in distributed systems, microservices, and API design.  




- Playwright is an open-source framework for cross-browser automation and 
  end-to-end web application testing. It was designed to be a fast, reliable, 
  robust, and evergreen test automation framework, and its API supports modern
  rendring engines that include Chromium, Firefox, WebKit, etc. ...
  



- A Zod schema is a TypeScript-first validation library used to define, validate
  , and parse data structures, ranging from simple strings to complex nested
  objects. It allows developers to declare a schema once and automatically
  infer static TypeScript types, eliminating duplicate code and ensuring runtime
  type safety.  


- Playwright for AI refers to the integration of Microsoft's browser automation
  tool with Artificial Intelligence to revolutionise end-to-end testing, 
  enabling autonomous test generation, self-healing, and intelligent interaction
  with web applications. using tools like Playwright MCP (Model Context Protocol
  ), ÅI agents can explore websites, click elements, and generate test scripts
  based on natural language commands.




- pnpm (Performant npm) is a fast, disk-space-efficient package manager for 
  Node.js that serves as a superior alternative to npm or yarn. It uses a 
  content-addressable store to save all package files in a single, global 
  location on a machine, using hard links and symlinks to reference them in
  projects, which eliminates redundant copies and speeds up installations.
      - Key Features && Advantages:
         - Disk Space Efficiency
         - Speed
         - Strict Dependencies
         - Monorepo Support: Built-in, robust support for monorepos (multiple
           projects in one repository) makes it ideal for complex, large-scale
           projects.
         - Drop-in Replacement



- Rolldown is a high-performance JavaScript/TypeScript bundler written in RUST,
  designed as the future unified engine for the Vite build tool. It aims to
  replace the current two-bundler setup in Vite (esbuild for development, Rollup
  for production) with a single, faster, and more efficient tool.

  KEY ASPECTS OF THE ROLLDOWN ENGINE:
  - Performance
  - Compatibility
  - ...
  - Vite Integration





- Vite (French for "quick") iks a next-generation, high-performance front-end
  build tool designed to make modern web development faster and leaner. It uses
  native ES modules in the browser for instant server start and provides 
  lightning-fast Hot Module Replacement (HMR), greatly improving developer 
  experience.

  KEY FEATURES && BENEFITS OF VITE:
     - instant server start
     - lightning fast HMR
     - optimised production build: uses rollup...
     - native ES module support
     - rich features: Built-in support for TS, JSX, CSS, and frameworks like 
       React, Vue, and Svelte




- Hot Module Replacement (HMR) is a development feature that updates, adds, or
  removes JS modules in a running application without requiring a full page
  reload. It enables near-instantaneous feedback by swapping only the modified
  code, significantly increasing development speed....




- Tsdown is a fast, specialised TypeScript/JavaScript library bundler built on
  the Rust-based `Rolldown` engine. It simplifies library development by 
  offering minimal configuration, automatic TS declaration (DTS) generation, and
  support for ESM/CJS formats. Often seen as a faster, modern successor to tsup,
  it is designed for building, testing, and publishing libraries with high 
  performance.



- E2E (End-to-End) software development covers the entoire software development
  lifecycle (SDLC), from initial planning and design to coding, deployment, and
  ongoing maintenance, typically managed by one team to ensure cohesion. It 
  streamlines processes, reduces costs, and accelerates delivery by removing
  dependency on multiple vendors.  



- A UUID (Universally Unique Identifier) is a 128-bit label used in computer
  systems to uniquely identify information, such as database rows, files, or 
  application sesssions. Represented as a 36-character alphanumeric string
  ... UUIDs are deisgned to be globally unique without a central registration 
  authority.